In [1]:
MAIN_PATH = "/home/carlos/MasterDS/tfm"
JSON_DATA_PATH = '{}/data/json/'.format(MAIN_PATH)
CSV_DATA_PATH = '{}/data/csv/'.format(MAIN_PATH)

In [2]:
import sys
sys.path.insert(0, MAIN_PATH)

In [10]:
%load_ext autoreload
%autoreload 2
from scripts.metrics.summary_evaluation import SummaryEvaluation
from scripts.models.rank_with_model.rank_with_model import RankModel
from scripts.models.baseline_rank.baseline_rank import BaselineRank
from scripts.models.ltr.train import LTRTrain
from scripts.extractive_summary.ltr.ltr_features_targets import LTRFeaturesTargets
from scripts.models.ltr_gbt.train import LTRGBTTrain
from scripts.models.ltr_svm_tf.train import LTRSVMTFTrain
from scripts.models.ltr_nn_tf.train import LTRNNTFTrain
from scripts import conf
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import pandas as pd

# Evaluación

Se usará el mismo desarrollo que se utilizó cuando se calcularon las métricas de los resúmenes generados por eventos clave y por grafos
conceptuales

# Baseline

## Carga de datos

In [20]:
key_events = ['goal', 'red_card', 'penalty']
lags = [1, 3, 5]
target_metric = 'cosine_tfidf'
drop_teams = True
lemma = True
metric_params = {'ngram_range': (1, 2), 'strip_accents': 'unicode'}
count_vec_kwargs = {'ngram_range': (1, 2), 'strip_accents': 'unicode'}

In [21]:
ltr = LTRFeaturesTargets(target_metric=target_metric, 
                        key_events=key_events,
                        lags=lags,
                        metric_params=metric_params,
                        count_vec_kwargs=count_vec_kwargs,
                        drop_teams=drop_teams,
                        lemma=lemma)

Setting target metric to cosine_tfidf


In [22]:
rank = BaselineRank(ltr=ltr, n=10)

Received LTR Features targets. Training and scoring are not necessary


In [23]:
rank.run()

Ranking events using row_number approach...
Saving to /home/carlos/MasterDS/tfm/models/baseline_rank/34e2c72e03/summaries.csv


In [24]:
rank.ltr.target_metric

'cosine_tfidf'

## Evaluación

In [25]:
evaluation = SummaryEvaluation(metric='rouge')

Setting target metric to rouge


### Sin procesar

In [26]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank,
                                                   preprocess_text=False)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/baseline_rank/34e2c72e03/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/baseline_rank_cosine_tfidf
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/baseline_rank_cosine_tfidf.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/baseline_rank_cosine_tfidf_avg.pickle


In [27]:
avg_scores_dict

{'rouge-1': {'f': 0.19979229468420182,
  'p': 0.24900335591362643,
  'r': 0.1948519608845315},
 'rouge-2': {'f': 0.043968174546395476,
  'p': 0.05707513269938273,
  'r': 0.04218564407361283},
 'rouge-l': {'f': 0.1885891367106406,
  'p': 0.3205694129713662,
  'r': 0.14408239323536431}}

### Con procesado

In [28]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank,
                                                   preprocess_text=True)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/baseline_rank/34e2c72e03/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/baseline_rank_cosine_tfidf_processed
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/baseline_rank_cosine_tfidf_processed.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/baseline_rank_cosine_tfidf_processed_avg.pickle


In [29]:
avg_scores_dict

{'rouge-1': {'f': 0.20614710520254323,
  'p': 0.2370105533948301,
  'r': 0.21777546369970727},
 'rouge-2': {'f': 0.06223472563933338,
  'p': 0.0705535173269253,
  'r': 0.06670498029156349},
 'rouge-l': {'f': 0.23652474459866485,
  'p': 0.37782913410707086,
  'r': 0.1873853769149725}}

# Random forest

## Carga de datos

In [87]:
ltr_params = {
    'key_events': ['goal', 'red_card', 'penalty'],
    'lags': [1, 3, 5],
    'target_metric': 'rouge',
    'drop_teams': True,
    'lemma': True,
    'metric_params': {'rouge_mode': 'rouge-1', 'rouge_metric': 'r'},
    'count_vec_kwargs': {'ngram_range': (1, 2), 'strip_accents': 'unicode'}
}
num_features = ['tfidf_sum']
cat_features_dict = {'is_key_event': [0, 1],
                     'n_players_cat': ['no_player', 'one_player', 'more_than_one_player']}

model_params = {
    'n_estimators': [100, 200, 500],
    'max_depth': [2, 5, 7],
    'bootstrap': [True],
    
}
cv = 5
opt_metric = 'neg_mean_squared_error'

In [20]:
ltr_train = LTRTrain(cat_features_dict=cat_features_dict, num_features=num_features,
                     model_params=model_params, ltr_params=ltr_params)

Setting target metric to cosine_tfidf


In [88]:
#CV
ltr_train = LTRTrain(cat_features_dict=cat_features_dict,
                     num_features=num_features,
                     model_params=model_params, ltr_params=ltr_params,
                    cv=cv, opt_metric=opt_metric)

Setting target metric to rouge


In [89]:
rank_model = RankModel(ltr_train, n=10)

Model already trained


## Evaluación

In [90]:
evaluation = SummaryEvaluation(metric='rouge')

Setting target metric to rouge


### Sin procesar

In [91]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                        preprocess_text=False)

Categorizing n_players...
Computing new length...
Dropping {'total_length', 'url', 'score', 'event_ix', 'n_players', 'advantage', 'players_importance', 'sim_previous_1', 'n_stop', 'sim_previous_5', 'sim_previous_3', 'equalize', 'json_file', 'sentence_ix', 'length', 'position'}
Ranking events using row_number approach...
Saving to /home/carlos/MasterDS/tfm/models/rank_with_model/1e1b5e1354/summaries.csv
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/1e1b5e1354/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_random_forest_rouge-1_r_4649644395
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_random_forest_rouge-1_r_4649644395.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_random_forest_rouge-1_r_4649644395_avg.pickle


In [92]:
avg_scores_dict

{'rouge-1': {'f': 0.20201338129646834,
  'p': 0.20561044053718983,
  'r': 0.23973303543285196},
 'rouge-2': {'f': 0.03974249089134354,
  'p': 0.041704530457269136,
  'r': 0.04666785731645699},
 'rouge-l': {'f': 0.19690542623267518,
  'p': 0.29621237217045404,
  'r': 0.15845738656779743}}

### Con procesado

In [93]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=True)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/1e1b5e1354/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_random_forest_rouge-1_r_4649644395_processed
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_random_forest_rouge-1_r_4649644395_processed.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_random_forest_rouge-1_r_4649644395_processed_avg.pickle


In [94]:
avg_scores_dict

{'rouge-1': {'f': 0.18425479134550568,
  'p': 0.178717023506525,
  'r': 0.23507950641128378},
 'rouge-2': {'f': 0.05364293652901957,
  'p': 0.051345724460198874,
  'r': 0.06983185816898395},
 'rouge-l': {'f': 0.24265352816778532,
  'p': 0.34579861320944005,
  'r': 0.20300152264615015}}

# GBT

## Carga de datos

In [5]:
ltr_params = {
    'key_events': ['goal', 'red_card', 'penalty'],
    'lags': [1, 3, 5],
    'target_metric': 'cosine_tfidf',
    'drop_teams': True,
    'lemma': True,
    'metric_params': {'ngram_range': (1, 2), 'strip_accents': 'unicode'},
    'count_vec_kwargs': {'ngram_range': (1, 2), 'strip_accents': 'unicode'}
}
cat_features_dict = {'is_key_event': [0, 1]}
num_features = ['tfidf_sum']
model_params = {
    'min_samples_leaf': [50],
    'subsample': [0.9],  
    'learning_rate': [0.1],
    'n_estimators': [80],
    'max_depth':[11],
    'min_samples_split': [200],
}
cv = 5
opt_metric = 'neg_mean_squared_error'

In [6]:
ltr_train = LTRGBTTrain(cat_features_dict=cat_features_dict, num_features=num_features,
                     model_params=model_params, ltr_params=ltr_params,
                    cv=cv, opt_metric=opt_metric)

Setting target metric to cosine_tfidf


In [7]:
rank_model = RankModel(ltr_train, n=10)

Model already trained


In [8]:
rank_model.path

'/home/carlos/MasterDS/tfm/models/rank_with_model/77752e3840'

## Evaluación

In [9]:
evaluation = SummaryEvaluation(metric='rouge')

Setting target metric to rouge


### Sin procesar

In [10]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=False)

Categorizing n_players...
Computing new length...
Dropping {'n_players', 'advantage', 'sim_previous_3', 'url', 'n_players_cat', 'equalize', 'position', 'sim_previous_1', 'players_importance', 'event_ix', 'sim_previous_5', 'json_file', 'sentence_ix', 'total_length', 'n_stop', 'score', 'length'}
Ranking events using row_number approach...
Saving to /home/carlos/MasterDS/tfm/models/rank_with_model/77752e3840/summaries.csv
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/77752e3840/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_gbt_cosine_tfidf_433610900a
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_gbt_cosine_tfidf_433610900a.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_gbt_cosine_tfidf_433610900a_avg.pickle


In [11]:
avg_scores_dict

{'rouge-1': {'f': 0.20595819542257637,
  'p': 0.21294021569065466,
  'r': 0.2380229271112617},
 'rouge-2': {'f': 0.04076413347530134,
  'p': 0.043481267381091995,
  'r': 0.046611681295501264},
 'rouge-l': {'f': 0.19683925177513747,
  'p': 0.29301893632591197,
  'r': 0.15973914366945902}}

### Con procesado

In [12]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=True)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/77752e3840/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_gbt_cosine_tfidf_433610900a_processed
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_gbt_cosine_tfidf_433610900a_processed.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_gbt_cosine_tfidf_433610900a_processed_avg.pickle


In [13]:
avg_scores_dict

{'rouge-1': {'f': 0.19242624571222655,
  'p': 0.18867534276289766,
  'r': 0.23972215301339264},
 'rouge-2': {'f': 0.05576108472336706,
  'p': 0.0539154301546138,
  'r': 0.07087355861129793},
 'rouge-l': {'f': 0.2455905442307586,
  'p': 0.34533623756262893,
  'r': 0.20749129238902364}}

# SVM

## Carga de datos

In [5]:
ltr_params = {
    'mode': 'tfidf',
    'count_vec_kwargs': {'strip_accents': 'unicode'},
    'target_metric': 'cosine_tfidf',
    'metric_params': {'ngram_range': (1, 2), 'strip_accents': 'unicode'},
    'drop_teams': True,
    'lemma': True,
    'train_perc': 0.7,
    'val_perc': 0.2
}

model_params = {}

In [6]:
ltr_train = LTRSVMTFTrain(model_params=model_params, ltr_params=ltr_params)

Setting target metric to cosine_tfidf
Setting mode to tfidf


In [7]:
rank_model = RankModel(ltr_train, n=10)

Model already trained


In [8]:
rank_model.path

'/home/carlos/MasterDS/tfm/models/rank_with_model/c842b51b47'

## Evaluación

In [9]:
evaluation = SummaryEvaluation(metric='rouge')

Setting target metric to rouge


### Sin procesar

In [10]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=False)

Ranking events using row_number approach...
Saving to /home/carlos/MasterDS/tfm/models/rank_with_model/c842b51b47/summaries.csv
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/c842b51b47/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_svm_cosine_tfidf_5959a3c0eb
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_svm_cosine_tfidf_5959a3c0eb.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_svm_cosine_tfidf_5959a3c0eb_avg.pickle


In [11]:
avg_scores_dict

{'rouge-1': {'f': 0.21529859629075385,
  'p': 0.24442658278516374,
  'r': 0.2277554166945007},
 'rouge-2': {'f': 0.04625120734018724,
  'p': 0.05450816217309867,
  'r': 0.0481403555868},
 'rouge-l': {'f': 0.20077112166486977,
  'p': 0.30833042485401896,
  'r': 0.16000087436398758}}

### Con procesado

In [12]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=True)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/c842b51b47/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_svm_cosine_tfidf_5959a3c0eb_processed
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_svm_cosine_tfidf_5959a3c0eb_processed.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_svm_cosine_tfidf_5959a3c0eb_processed_avg.pickle


In [13]:
avg_scores_dict

{'rouge-1': {'f': 0.21505350047542593,
  'p': 0.22811291380978138,
  'r': 0.24578301127756888},
 'rouge-2': {'f': 0.06132902014201187,
  'p': 0.06391827815574572,
  'r': 0.0715563001615982},
 'rouge-l': {'f': 0.25628513900656386,
  'p': 0.3721991832613804,
  'r': 0.21213475574391005}}

# NN

## Carga de datos

In [99]:
ltr_params = {
    'mode': 'tfidf',
    'count_vec_kwargs': {'strip_accents': 'unicode'},
    'target_metric': 'cosine_tfidf',
    'metric_params': {'ngram_range': (1, 2), 'strip_accents': 'unicode'},
    'drop_teams': True,
    'lemma': True,
    'train_perc': 0.7,
    'val_perc': 0.2
}
DENSE_LAYERS = [500, 250, 125, 60, 20]
N_DROPOUT = 4

model_params = {
    'dense_layers': DENSE_LAYERS,
    'dense_activation': ['relu'] * len(DENSE_LAYERS),
    'dropout': [0.3] * N_DROPOUT,
    'optimizer': 'adam'
}
opt_metric = 'mean_squared_error'
epochs = 25
batch_size = 128
# Not in config
shuffle = True
max_queue_size = 50

In [100]:
ltr_train = LTRNNTFTrain(ltr_params, epochs=epochs, batch_size=batch_size,
                         model_params=model_params,
                         opt_metric=opt_metric)

Adding 1 zeros so that params have the same length
Setting target metric to cosine_tfidf
Setting mode to tfidf
Optimizing mean_squared_error with adam
Epochs: 25
Batch size: 128


In [101]:
rank_model = RankModel(ltr_train, n=10, is_nn=True)

Model already trained


In [15]:
rank_model.path

'/home/carlos/MasterDS/tfm/models/rank_with_model/254dd4d848'

## Evaluación

In [102]:
evaluation = SummaryEvaluation(metric='rouge')

Setting target metric to rouge


### Sin procesar

In [103]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=False)

Ranking events using row_number approach...
Saving to /home/carlos/MasterDS/tfm/models/rank_with_model/6f4ca361f3/summaries.csv
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/6f4ca361f3/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_nn_cosine_tfidf_598b6e8db8
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_nn_cosine_tfidf_598b6e8db8.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_nn_cosine_tfidf_598b6e8db8_avg.pickle


In [104]:
avg_scores_dict

{'rouge-1': {'f': 0.2121848576828116,
  'p': 0.23666642017257633,
  'r': 0.2307449150710787},
 'rouge-2': {'f': 0.04580426040500962,
  'p': 0.05305677671571103,
  'r': 0.04897908854164727},
 'rouge-l': {'f': 0.20152933004411733,
  'p': 0.30717488321505537,
  'r': 0.161272672149186}}

### Con procesado

In [105]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=True)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/6f4ca361f3/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_nn_cosine_tfidf_598b6e8db8_processed
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_nn_cosine_tfidf_598b6e8db8_processed.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/rank_with_model_ltr_nn_cosine_tfidf_598b6e8db8_processed_avg.pickle


In [106]:
avg_scores_dict

{'rouge-1': {'f': 0.20929769829227252,
  'p': 0.21985300131783983,
  'r': 0.24450120365736216},
 'rouge-2': {'f': 0.05989609131129074,
  'p': 0.06175092321404012,
  'r': 0.07150376598052609},
 'rouge-l': {'f': 0.25569183133753215,
  'p': 0.37070862481839606,
  'r': 0.2120668823777382}}

# Eventos clave y grafos conceptuales

In [75]:
%%time
evaluation.evaluate_all_summaries()

Evaluating the following summaries: ['key_events_summaries_1.csv', 'key_events_summaries_3.csv', 'key_events_summaries_4.csv', 'key_events_summaries_graph_2.csv', 'key_events_summaries_graph_3.csv', 'key_events_summaries_2.csv', 'key_events_summaries_graph_5.csv', 'key_events_summaries_graph_1.csv', 'key_events_summaries_graph_4.csv']
Evaluating key_events_summaries_1.csv
Metrics already exist
Evaluating key_events_summaries_3.csv
Metrics already exist
Evaluating key_events_summaries_4.csv
Metrics already exist
Evaluating key_events_summaries_graph_2.csv
Metrics already exist
Evaluating key_events_summaries_graph_3.csv
Metrics already exist
Evaluating key_events_summaries_2.csv
Metrics already exist
Evaluating key_events_summaries_graph_5.csv
Metrics already exist
Evaluating key_events_summaries_graph_1.csv
Metrics already exist
Evaluating key_events_summaries_graph_4.csv
Metrics already exist
CPU times: user 1.88 s, sys: 122 ms, total: 2 s
Wall time: 3.86 s


# Todos los resúmenes

In [18]:
pd_metrics = evaluation.output_avg_metrics()

In [19]:
pd.pivot_table(pd_metrics, index='experiment', columns=['metric', 'metric_type'], values='value')

metric                                               rouge-1            \
metric_type                                                f         p   
experiment                                                               
baseline_rank_rouge-1_r                             0.217420  0.244630   
baseline_rank_rouge-1_r_processed                   0.214708  0.221810   
key_events_summaries_1                              0.152305  0.337611   
key_events_summaries_1_processed                    0.195851  0.520812   
key_events_summaries_2                              0.151979  0.338229   
key_events_summaries_2_processed                    0.195232  0.522998   
key_events_summaries_3                              0.152305  0.337611   
key_events_summaries_3_processed                    0.195851  0.520812   
key_events_summaries_4                              0.151979  0.338229   
key_events_summaries_4_processed                    0.195232  0.522998   
key_events_summaries_graph_1                        0.176339  0.155282   
key_events_summaries_graph_1_processed              0.131490  0.107596   
key_events_summaries_graph_2                        0.151728  0.120522   
key_events_summaries_graph_2_processed              0.122147  0.091738   
key_events_summaries_graph_3                        0.176339  0.155282   
key_events_summaries_graph_3_processed              0.131490  0.107596   
key_events_summaries_graph_4                        0.151728  0.120522   
key_events_summaries_graph_4_processed              0.122147  0.091738   
key_events_summaries_graph_5                        0.173270  0.140934   
key_events_summaries_graph_5_processed              0.134658  0.101599   
rank_with_model_ltr_gbt_cosine_tfidf_433610900a     0.205958  0.212940   
rank_with_model_ltr_gbt_cosine_tfidf_433610900a...  0.192426  0.188675   
rank_with_model_ltr_gbt_rouge-1_r_be3b14bd22        0.202762  0.207199   
rank_with_model_ltr_gbt_rouge-1_r_be3b14bd22_pr...  0.186269  0.181213   
rank_with_model_ltr_nn_cosine_tfidf_598b6e8db8      0.212185  0.236666   
rank_with_model_ltr_nn_cosine_tfidf_598b6e8db8_...  0.209298  0.219853   
rank_with_model_ltr_nn_rouge-1_r_4083eddade         0.214111  0.224458   
rank_with_model_ltr_nn_rouge-1_r_4083eddade_pro...  0.203637  0.199008   
rank_with_model_ltr_random_forest_cosine_tfidf_...  0.203121  0.206317   
rank_with_model_ltr_random_forest_cosine_tfidf_...  0.184689  0.178371   
rank_with_model_ltr_random_forest_rouge-1_r_464...  0.202013  0.205610   
rank_with_model_ltr_random_forest_rouge-1_r_464...  0.184255  0.178717   
rank_with_model_ltr_svm_cosine_tfidf_5959a3c0eb     0.215299  0.244427   
rank_with_model_ltr_svm_cosine_tfidf_5959a3c0eb...  0.215054  0.228113   
rank_with_model_ltr_svm_rouge-1_r_8dd264b4d2        0.217884  0.229743   
rank_with_model_ltr_svm_rouge-1_r_8dd264b4d2_pr...  0.209460  0.205204   
upper_bound                                         0.105305  0.067818   

metric                                                         rouge-2  \
metric_type                                                r         f   
experiment                                                               
baseline_rank_rouge-1_r                             0.225864  0.046198   
baseline_rank_rouge-1_r_processed                   0.243826  0.060661   
key_events_summaries_1                              0.120764  0.037226   
key_events_summaries_1_processed                    0.152725  0.062947   
key_events_summaries_2                              0.120376  0.037130   
key_events_summaries_2_processed                    0.151988  0.062645   
key_events_summaries_3                              0.120764  0.037226   
key_events_summaries_3_processed                    0.152725  0.062947   
key_events_summaries_4                              0.120376  0.037130   
key_events_summaries_4_processed                    0.151988  0.062645   
key_events_summaries_graph_1                        0.251348  0.035792   
key

In [14]:
evaluation.bound_metrics()

Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/upper_bound.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/rouge/upper_bound_avg.pickle


In [26]:
pd_bound = evaluation.output_avg_bound()

In [27]:
pd_bound

,metric,metric_type,value
0,rouge-1,f,0.105305
1,rouge-1,p,0.067818
2,rouge-1,r,0.361353
3,rouge-2,f,0.021309
4,rouge-2,p,0.013826
5,rouge-2,r,0.073771
6,rouge-l,f,0.218348
7,rouge-l,p,0.228750
8,rouge-l,r,0.232812
